In [4]:
# Identify Data types of number of data in each column 

In [39]:
import pandas as pd

df = pd.read_csv("https://datasocibmproject.s3.ap-southeast-2.amazonaws.com/structured_data/capital_project_schedules_and_budgets_1.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8185 entries, 0 to 8184
Data columns (total 16 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Unnamed: 0                                                  8185 non-null   int64  
 1   project_geographic_district                                 8185 non-null   int64  
 2   project_building_identifier                                 8185 non-null   object 
 3   project_school_name                                         8185 non-null   object 
 4   project_type                                                8185 non-null   object 
 5   project_description                                         8185 non-null   object 
 6   project_phase_name                                          8184 non-null   object 
 7   project_status_name                                         8185 non-null   object 
 8 

In [6]:
df['project_geographic_district'].min()

1

In [7]:
df['project_geographic_district'].min()

1

In [8]:
df['project_status_name'].head()

0    In-Progress
1    In-Progress
2       Complete
3    In-Progress
4            PNS
Name: project_status_name, dtype: object

In [9]:
for index in df.duplicated():
    if index == True:
        print(index) 

# Checking Wrong data inputs for date

In [10]:
def checkDataFormatting(df, col):
    wrong_format_count = 0
    unique_value_wrong_format_dict = {}

    for date in df[col]:
        #correct = False
        correct_format = checkLenghtdata(date)
        if correct_format == False:
            # input directionary 
            unique_value_wrong_format_dict = uniqueWronglyFormatedDatas(unique_value_wrong_format_dict, date)
            wrong_format_count += 1
            
    return wrong_format_count, unique_value_wrong_format_dict

def checkLenghtdata(data):
    correct_format = False
    # convert string
    data = str(data)
    # CHeck if data is in form XX/XX/XXXX or PNS 
    if len(data) == 10 or data == 'PNS' or data == 'F&E' or data == 'CM':
        correct_format = True
        
        
    return correct_format

def uniqueWronglyFormatedDatas(dictionary, date):
    if date in dictionary:
        # add new key and dictionary 
        dictionary[date] += 1 
    else:
        dictionary[date] = 1
    return dictionary



In [11]:
# is the date consistent? in the form XX / XX / XXXX

wrong_actual_date_count_start = 0
wrong_actual_unique_dict_start = {}

wrong_actual_date_count_end = 0
wrong_actual_unique_dict_end = {}

wrong_planned_date_count_end = 0
wrong_planned_unique_dict_end = {}


wrong_actual_date_count_start, wrong_actual_unique_dict_start = checkDataFormatting(df, 'project_phase_actual_start_date')
wrong_actual_date_count_end, wrong_actual_unique_dict_end = checkDataFormatting(df,'project_phase_actual_end_date')
wrong_planned_date_count_end, wrong_planned_unique_dict_end = checkDataFormatting(df, 'project_phase_planned_end_date')

print(f"For project_phase_actual_start_date there is total wrongly formated {wrong_actual_date_count_start} and unique wrong values includes {wrong_actual_unique_dict_start} \n")
print(f"For project_phase_actual_end_date there is total wrongly formated {wrong_actual_date_count_end} and unique wrong values includes {wrong_actual_unique_dict_end} \n")
print(f"For project_phase_planned_end_date there is total wrongly formated {wrong_planned_date_count_end} and unique wrong values includes {wrong_planned_unique_dict_end} \n")



For project_phase_actual_start_date there is total wrongly formated 1 and unique wrong values includes {'FTK': 1} 

For project_phase_actual_end_date there is total wrongly formated 2159 and unique wrong values includes {nan: 2158, 'FTK': 1} 

For project_phase_planned_end_date there is total wrongly formated 1324 and unique wrong values includes {'DOES': 318, 'IEH': 652, 'DOEL': 113, 'DOER': 170, 'FTK': 61, 'EMER': 4, 'DOEP': 1, '/  /': 5} 



In [12]:
# Checking which index have null or Nan values
df.items()

<generator object DataFrame.items at 0x12701e3e0>

In [13]:
import numpy as np 
import json  
wrong_missing_dictionary = {}
# Iterate through DataFrame columns
for column in df:
    wrong_missing_dictionary[column] = ''
    # Iterate through indices of the column
    for value in df[column]:
        # Checks for NaN, empty string (''), or None
        if pd.isna(value) or value == '' or value == None:
            wrong_missing_dictionary[column] = [value, index]

print(json.dumps(wrong_missing_dictionary, indent=4))

{
    "Unnamed: 0": "",
    "project_geographic_district": "",
    "project_building_identifier": "",
    "project_school_name": "",
    "project_type": "",
    "project_description": "",
    "project_phase_name": [
        NaN,
        false
    ],
    "project_status_name": "",
    "project_phase_actual_start_date": "",
    "project_phase_planned_end_date": "",
    "project_phase_actual_end_date": [
        NaN,
        false
    ],
    "project_budget_amount": "",
    "final_estimate_of_actual_costs_through_end_of_phase_amount": [
        NaN,
        false
    ],
    "total_phase_actual_spending_amount": "",
    "dsf_number_s": "",
    "failure": ""
}


# Finding Duplicates

In [14]:

duplicated_indices = []
print(len(df))
index = 0
for value in df.duplicated():
    if  value == True:
        duplicated_indices.append(index)
    index += 1


print(duplicated_indices)

8185
[]


# Finding Range of Unique Values in each column

In [5]:
# Goals is to find all unique entries in each column 
# if it is str then puts all numbers eahc unique entry into string
# if is it int() then finds range (min and max) else if does not int then puts idnex and row
def uniqueEntriesFeature(df, col):
    min_entry = df[col][1]
    max_entry = 0
    unique_dictionary = {} 
        
    min_key = f"{col}_max"
    max_key = f"{col}_max"
    unique_key = f"{col}_unique"
    
    unique_dictionary[min_key] = min_entry
    unique_dictionary[max_key] = max_entry
    unique_dictionary[unique_key] = []
    
       
    for row, entry in enumerate(df[col]):
        try:
            # Convert entry to float (assuming numeric data)
            value = int(entry) 
            if value < min_entry:
                unique_dictionary[min_key] = value
                min_entry = value
            elif value > max_entry:
                unique_dictionary[max_key] = value
                max_entry = value
                
        except ValueError:
            # Handle non-numeric entries (strings, NaNs, etc.)
            if entry not in unique_dictionary[unique_key]:
                unique_dictionary[unique_key].append(entry)

    return unique_dictionary
    
                

unique_dict = uniqueEntriesFeature(df, 'project_geographic_district')
print(unique_dict)


{'project_geographic_district_max': 32, 'project_geographic_district_unique': []}


In [6]:
unique_dict = uniqueEntriesFeature(df, 'project_geographic_district')
print(unique_dict)
df['project_geographic_district'].unique()

{'project_geographic_district_max': 32, 'project_geographic_district_unique': []}


array([10,  2, 11,  3, 25,  4, 15,  5, 16,  6, 12, 13,  7,  8, 14,  9,  1,
       18, 30, 24, 19, 31, 20, 26, 21, 27, 22, 28, 23, 29, 17, 32])

In [34]:
df['dsf_number_s'].unique()

array(['DSF: 0000821031, 0000822210', 'DSF: 0000823560',
       'DSF: 0000844624', ...,
       'DSF: 0000822169, 0000843221, 0000843222, 0000843223',
       'DSF: 0000851654', 'DSF: 0000776207'], dtype=object)

In [56]:
unique_dict = uniqueEntriesFeature(df, 'project_school_name')
#print(unique_dict)
row_to_drop = [] 
values = unique_dict['project_school_name_unique']
for row, value in enumerate(values):
    if value.lower().strip() in ['/','//','nan'] or pd.isnull(value) or pd.isna(value):
        row_to_drop.append(row)


row_to_drop

[]

In [46]:
# find if there is problematric values for project_building_identifier
import re 
# Method 1
arr_project_building_identifier = df['project_building_identifier'].unique()
wrong_input_arr_project_building_identifier = {}
for index, building_identifer in enumerate(df['project_building_identifier']):
    withoutletter_building_identifer = building_identifer[1:len(building_identifer)]
    try:
        withoutletter_building_identifer = building_identifer[1:len(building_identifer)]
        withoutletter_building_identifer = int(withoutletter_building_identifer)
        if building_identifer[0].lower().isalpha() == False or len(building_identifer) != 4 :
            wrong_input_arr_project_building_identifier[building_identifer] = index
    except ValueError as e:
        wrong_input_arr_project_building_identifier[building_identifer] = index

#wrong_input_arr_project_building_identifier

# Method 2 - using regualar expressions to find porblematic values
expected_pattern = r'^[A-Z\. ][0-9][0-9][0-9]$'
wrong_input_arr_project_building_identifier_method_two = {}
for index, value in enumerate(df['project_building_identifier']):
    value.strip()
    if not re.match(expected_pattern, value):
        wrong_input_arr_project_building_identifier_method_two[value] = index
        
print(wrong_input_arr_project_building_identifier_method_two)

{'XAAF': 1479, 'XADJ': 1480, 'XADO': 1481, 'XADY': 1482, 'XAEE': 1483, 'XABI': 2606, 'XAEO': 2607, 'XAFB': 2608, 'XAUJ': 2609, 'QALP': 5504, 'QAPM': 5505, 'QMAD': 6089, 'KBZT': 6221, 'KAKB': 6912}


In [36]:
import numpy as np
np.shape(df)
print(df.iloc[1479,:])
print(np.shape(df))
df.drop(labels=[1480], axis=1,inplace=True)
print(np.shape(df))

Unnamed: 0                                                                      1480
project_geographic_district                                                       11
project_building_identifier                                                     XADJ
project_school_name                                                     X000 - BRONX
project_type                                                    SCA Furniture & Equi
project_description                                           FY16 RESO A TECHNOLOGY
project_phase_name                                                               F&E
project_status_name                                                      In-Progress
project_phase_actual_start_date                                           01/09/2017
project_phase_planned_end_date                                                   F&E
project_phase_actual_end_date                                                    NaN
project_budget_amount                                            

In [ ]:
expected_pattern = r'^[A-Z\. ]+\s*-\s*[A-Z]+$'

In [37]:
df['project_school_name'].unique()

array(['I.S. 254 - BRONX', 'P.S. 279 - BRONX', 'I.S. 228 - BRONX', ...,
       'ACAD. FOR ENVIRON. LDRSHP - K', 'P.S. 677 - BROOKLYN',
       'ALC - BROOKLYN CHOICES'], dtype=object)

# project_school_name
Since project description takes on mainly two forms I.S. 254 - BRONX or ALC - BROOKLYN CHOICES  or ACAD. FOR ENVIRON. LDRSHP - K
our main goals is to checks if the expression includes 2 or more characters and a dash
From checks all seem fine 

In [47]:
df['project_school_name'].unique()

array(['I.S. 254 - BRONX', 'P.S. 279 - BRONX', 'I.S. 228 - BRONX', ...,
       'ACAD. FOR ENVIRON. LDRSHP - K', 'P.S. 677 - BROOKLYN',
       'ALC - BROOKLYN CHOICES'], dtype=object)

In [48]:
dic_wrong_project_school_name = {}
reg_project_school_name = r'^(?=.*[A-Za-z].*[A-Za-z].*[A-Za-z])(?=.*-.*$).*$'
for index,value in enumerate(df['project_school_name']):
    if not re.match(reg_project_school_name, value):
        dic_wrong_project_school_name[value] = index
print(dic_wrong_project_school_name)

{'HS FOR ENVIRONMENTAL STUDIES': 154, 'OLD MABLE DEAN BACON VOC HS AX': 131, 'HARVEST COLLEGIATE HIGH SCHOOL': 161, 'HS FOR LEADERSHIP & PUBLIC SER': 165, 'CITY KNOLL MIDDLE SCHOOL': 168, 'LYFE': 213, 'M.S. M247 DUAL LANGUAGE MIDDLE': 228, 'WADLEIGH SCNDRY SCHL FOR PERMG': 244, 'WADLEIGH SECONDARY SCHOOL FOR': 370, 'HIGH SCHOOL FOR ENERGY AND TEC': 256, 'SCHOOL OF DIPLOMACY': 363, 'THE RIVERSIDE SCHOOL FOR MAKER': 477, 'URBAN ASSEMBLY SCHOOL': 1110, 'PELHAM ACADEMY OF ACADEMICS AN': 525, 'COLLEGIATE INST FOR MATH & SCI': 583, 'CORNERSTONE ACADEMY FOR SOCIAL': 567, 'YOUNG VOICES ACADEMY OF THE BR': 572, 'ACADEMY FOR SCHOLARSHIP AND EN': 576, 'PELHAM HIGH SCHOOL FOR LANGUAG': 584, 'HS OF COMPUTERS & TECHNOLOGY': 595, 'INNOVATION DIPLOMA PLUS': 644, 'THE URBAN ASSEMBLY SCHOOL': 1223, 'THE GLOBAL LEARNING COLLABORAT': 645, 'MAXINE GREEN HIGH SCHOOL': 670, 'COAL. SCHOOL FOR SOCIAL CHANGE': 723, 'SUMMIT ACADEMY CHARTER SCHOOL': 740, 'RED HOOK NEIGHBORHOOD SCHOOL': 742, 'P.S. 032 SAMUEL MILLS

df['project_type'].unique()

# Project_type

In [52]:
df['project_type'].unique()

array(['SCA CIP', 'SCA CIP RESOA', 'DIIT - RESOA', 'DOE- RESOA',
       'SCA Furniture & Equi', 'Trust For Public Lan', 'SCA Capacity',
       'DOE - Skilled Trades', 'SCA IEH', 'DOE - Lead Paint', 'PRE-K',
       'DIIT - PBX Telephone', 'DIIT - Project Conne',
       'Fast Track Projects', 'DCAS/NYPA', 'SCA Lease Site Impro',
       'SCA Emergency Lighti', 'RCT CIP', 'SCA Emergency Respon',
       'DOE Managed PREK'], dtype=object)

In [56]:
arr_na = df['project_type'].isna()
arr_project_type_na = {}
for index, value in enumerate(arr_na):
    if value == True:
        arr_project_type_na[index] = value

print(arr_project_type_na)

{}


In [58]:
arr_null_project_type = df['project_type'].isnull()
arr_null_project_type.unique()

array([False])

In [64]:
# Project description 
arr_null_description_type = df['project_description'].isnull().unique()
arr_na_description_type = df['project_description'].isna().unique()
print(arr_null_description_type)
print(arr_na_description_type)

[False]
[False]


# Project_phase

# Project Phase

In [65]:
print(df['project_phase_name'].unique())

['Construction' 'CM, F&E' 'Scope' 'Design' 'Purch & Install' 'F&E'
 'CM,Art,F&E' 'CM' nan]


# Project_status

In [66]:
df['project_status_name'].unique()

array(['In-Progress', 'Complete', 'PNS'], dtype=object)

In [67]:
df.iloc[6470, :]

Unnamed: 0                                                                                    6470
project_geographic_district                                                                     28
project_building_identifier                                                                   Q140
project_school_name                                                              P.S. 140 - QUEENS
project_type                                                                  Trust For Public Lan
project_description                                           FY17 RESO A PLAYGROUND REDEVELOPMENT
project_phase_name                                                                          Design
project_status_name                                                                       Complete
project_phase_actual_start_date                                                         10/21/2016
project_phase_planned_end_date                                                          10/15/2016
project_ph